In [ ]:
#Vizualize demand spatial distribution in Amsterdam emoped pilot

In [ ]:
### Imports 
import os
import math
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.patches as patches

import pandas as pd
import re
import xml.etree.ElementTree as ET

sys.path.append('../../')
from script.conversion.bison.coordinates import rd_to_utm
from mnms.graph.layers import PublicTransportLayer, MultiLayerGraph, OriginDestinationLayer
from mnms.generation.roads import generate_pt_line_road, generate_one_zone
from mnms.generation.layers import generate_bbox_origin_destination_layer
from mnms.vehicles.veh_type import Tram, Metro, Bus
from mnms.generation.zones import generate_one_zone
from mnms.mobility_service.public_transport import PublicTransportMobilityService
from mnms.time import TimeTable, Dt, Time
from mnms.io.graph import load_graph, save_graph, save_odlayer
from mnms.tools.render import draw_roads, draw_line, draw_odlayer
from mnms.tools.geometry import points_in_polygon, get_bounding_box

In [ ]:
### Parameters

# Files and directories
current_dir = os.getcwd()
indir = current_dir + '/inputs/'
outdir = current_dir + '/outputs/'

coord_csv_filepath = indir + 'KV1_GVB_2609_2/Csv/POINT.csv' # file with coordinates of the network
amsterdam_json_filepath = indir + 'new_network.json' # mlgraph with the road network only
ams_dmd_path = indir + 'test_all_in_highway_7h_9h.csv' # dmd inside ringroad 7-9am

In [ ]:
### Get the MLGraph without TCs 
amsterdam_graph = load_graph(amsterdam_json_filepath)
roads = amsterdam_graph.roads

In [ ]:
# Load dmd
db_dmd = pd.read_csv(ams_dmd_path, sep=';')

In [ ]:
db_dmd

In [ ]:
nb_dmd = len(db_dmd)
origins = np.zeros((len(db_dmd),2))
destinations = np.zeros((len(db_dmd),2))
for i, row in db_dmd[:].iterrows():
    origins[i] = [float(o) for o in row['ORIGIN'].split(' ')]
    destinations[i] = [float(d) for d in row['DESTINATION'].split(' ')]

In [ ]:
# Emoped initial locations
nb_emoped = 100

x = np.random.random(nb_emoped)*(origins[:,0].max()-origins[:,0].min()) + origins[:,0].min()
y = np.random.random(nb_emoped)*(origins[:,1].max()-origins[:,1].min()) + origins[:,1].min()
df_emoped = pd.DataFrame({'x':x, 'y':y})

In [ ]:
nodes_pos = np.array([node.position for node in roads.nodes.values()])
nodes_id = list(roads.nodes.keys())

In [ ]:
closest_nodes = []
x_nodes = []
y_nodes = []
for i, row in df_emoped.iterrows():
    x = row['x']
    y = row['y']
    dist_nodes = (x-nodes_pos[:,0])**2 + (y-nodes_pos[:,1])**2
    i_min = np.argmin(dist_nodes)
    closest_nodes.append(nodes_id[i_min])
    x_nodes.append(nodes_pos[i_min,0])
    y_nodes.append(nodes_pos[i_min,1])
df_emoped['closest_node'] = closest_nodes
df_emoped['x_node'] = x_nodes
df_emoped['y_node'] = y_nodes

In [ ]:
df_emoped

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
draw_roads(ax, roads, color='grey', linkwidth=0.1, nodesize=0, draw_stops=False, node_label=False)

plt.plot(origins[:,0], origins[:,1], 'b.', alpha=0.5)
plt.plot(destinations[:,0], destinations[:,1], 'r.', alpha=0.5)
#plt.plot(od_clustered[:,0], od_clustered[:,1], 'gd', alpha=0.5)

plt.plot(df_emoped['x_node'], df_emoped['y_node'],'gd')
plt.plot(df_emoped['x'], df_emoped['y'],'kd')
    
plt.xlim([624e3, 635e3])
plt.ylim([5.795e6, 5.8125e6])